In [1]:
# Import libraries
import pandas as pd

In [2]:
deptinfo_df = pd.read_csv(r'../Dillards POS/deptinfo.csv', header = None)
deptinfo_df = deptinfo_df.drop(columns=deptinfo_df.columns[-1])
deptinfo_df.columns = ['DEPT', 'DEPTDESC']
for i, v in enumerate(deptinfo_df.dtypes):
    if v not in [int, float]:
        deptinfo_df.iloc[:, i] = deptinfo_df.iloc[:, i].apply(lambda x: x.strip() if isinstance(x, str) else x)
print(deptinfo_df.head())
deptinfo_df.to_csv(r'../Dillards POS/clean_data/deptinfo_clean.csv', index=False)

   DEPT  DEPTDESC
0   800  CLINIQUE
1   801    LESLIE
2  1100    GARY F
3  1107   JACQUES
4  1202    CABERN


In [3]:
strinfo_df = pd.read_csv(r'../Dillards POS/strinfo.csv', header = None)
strinfo_df = strinfo_df.drop(columns=strinfo_df.columns[-1])
strinfo_df.columns = ['STORE', 'CITY', 'STATE', 'ZIP']
for i, v in enumerate(strinfo_df.dtypes):
    if v not in [int, float]:
        strinfo_df.iloc[:, i] = strinfo_df.iloc[:, i].apply(lambda x: x.strip() if isinstance(x, str) else x)
print(strinfo_df.head())
strinfo_df.to_csv(r'../Dillards POS/clean_data/strinfo_clean.csv', index=False)

   STORE            CITY STATE    ZIP
0      2  ST. PETERSBURG    FL  33710
1      3       ST. LOUIS    MO  63126
2      4     LITTLE ROCK    AR  72201
3      7      FORT WORTH    TX  76137
4      9           TEMPE    AZ  85281


In [4]:
skuinfo_df = pd.read_csv(r'../Dillards POS/skuinfo.csv', sep = "delimiter", header = None)
skuinfo_df_clean = skuinfo_df[skuinfo_df[0].apply(lambda x: x.count(',') == 10)]
skuinfo_df_clean = skuinfo_df_clean[0].str.split(',', expand=True)
skuinfo_df_clean = skuinfo_df_clean.drop(columns=skuinfo_df_clean.columns[-1])

/tmp/ipykernel_57541/1637673031.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  skuinfo_df = pd.read_csv(r'../Dillards POS/skuinfo.csv', sep = "delimiter", header = None)


In [5]:
# Change data types
skuinfo_df_clean[0] = skuinfo_df_clean[0].astype(int)
skuinfo_df_clean[1] = skuinfo_df_clean[1].astype(int)
skuinfo_df_clean[7] = skuinfo_df_clean[7].astype(int)
for i, v in enumerate(skuinfo_df_clean.dtypes):
    if v not in [int, float]:
        skuinfo_df_clean.iloc[:, i] = skuinfo_df_clean.iloc[:, i].apply(lambda x: x.strip() if isinstance(x, str) else x)
print(skuinfo_df_clean.dtypes)

0     int64
1     int64
2    object
3    object
4    object
5    object
6    object
7     int64
8    object
9    object
dtype: object


In [6]:
skuinfo_df_clean.columns = ['SKU', 'DEPT', 'CLASSID', 'UPC', 'STYLE', 'COLOR', 'SIZE', 'PACKSIZE', 'VENDOR', 'BRAND']
skuinfo_df_clean.to_csv(r'../Dillards POS/clean_data/skuinfo_clean.csv', index=False)

In [7]:
skstinfo_df = pd.read_csv(r'../Dillards POS/skstinfo.csv', header = None)
skstinfo_df = skstinfo_df.drop(columns=skstinfo_df.columns[-1])
skstinfo_df.columns = ['SKU', 'STORE', 'COST', 'RETAIL']
print(skstinfo_df.dtypes)

SKU         int64
STORE       int64
COST      float64
RETAIL    float64
dtype: object


In [8]:
duplicates = skstinfo_df[skstinfo_df.duplicated(subset=['SKU', 'STORE'], keep=False)]
len(duplicates)

0

In [9]:
# Make sure 'SKU' in the 'skstinfo' table is present in the referenced table
skstinfo_df_clean = skstinfo_df[skstinfo_df['SKU'].isin(skuinfo_df_clean['SKU'])]
# Make sure 'STORE' in the 'skstinfo' table is present in the referenced table
skstinfo_df_clean = skstinfo_df_clean[skstinfo_df_clean['STORE'].isin(strinfo_df['STORE'])]
skstinfo_df_clean.to_csv(r'../Dillards POS/clean_data/skstinfo_clean.csv', index=False)

In [10]:
import psycopg2
import getpass
import pandas.io.sql as psql

password = getpass.getpass("Please provide your password: ")
conn = psycopg2.connect(
    host="pg.analytics.northwestern.edu",
    database="everything2023",
    user="bvt1891", # change here!
    password=password)
print("Connected.")
cur = conn.cursor()

Please provide your password: ········
Connected.


In [11]:
query = """SELECT * from group_4.deptinfo"""
df = psql.read_sql(query,conn)
df.head()

/tmp/ipykernel_57541/4163816161.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query,conn)


,dept,deptdesc
0,800,CLINIQUE
1,801,LESLIE
2,1100,GARY F
3,1107,JACQUES
4,1202,CABERN


In [12]:
query = """SELECT * from group_4.trnsact LIMIT 100"""
df = psql.read_sql(query,conn)
df.head()

/tmp/ipykernel_57541/636336960.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query,conn)


,sku,store,register,trannum,interid,saledate,stype,quantity,orgprice,amt,seq,mic
0,3,202,290,1100,326708721,2005-01-18,P,1,0.0,30.0,3500000,818
1,3,202,540,2700,326708721,2005-01-29,R,1,0.0,30.0,15200000,818
2,3,303,500,2100,23702074,2004-08-18,P,1,0.0,12.0,4600000,48
3,3,709,360,500,0,2005-08-14,P,1,0.0,30.0,6500000,818
4,3,802,660,400,0,2005-08-09,P,1,440.0,30.0,4700000,599


In [13]:
query = """SELECT * from group_4.strinfo LIMIT 100"""
df = psql.read_sql(query,conn)
df.head()

/tmp/ipykernel_57541/3918319790.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query,conn)


,store,city,state,zip
0,2,ST. PETERSBURG,FL,33710
1,3,ST. LOUIS,MO,63126
2,4,LITTLE ROCK,AR,72201
3,7,FORT WORTH,TX,76137
4,9,TEMPE,AZ,85281


In [14]:
query = """SELECT * from group_4.skstinfo LIMIT 100"""
df = psql.read_sql(query,conn)
df.head()

/tmp/ipykernel_57541/1160443256.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query,conn)


,sku,store,cost,retail
0,3,102,123.36,440.0
1,3,103,123.36,440.0
2,3,104,123.36,440.0
3,3,202,123.36,440.0
4,3,203,123.36,440.0


In [15]:
query = """SELECT * from group_4.skuinfo LIMIT 100"""
df = psql.read_sql(query,conn)
df

/tmp/ipykernel_57541/1530076766.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query,conn)


,sku,dept,classid,upc,style,color,size,packsize,vendor,brand
0,3,6505,113,000400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,000400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,000400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,000400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,000400000015000,126 MDU461,255CAMEL,12,1,0023272,JONES/LA
...,...,...,...,...,...,...,...,...,...,...
95,422,801,204,000400000422000,09 5280M7,013BLACK/WHI,14,1,1616222,TAHARI/A
96,423,2102,973,000400000423000,R524,VANILLA,38D,1,9520439,DIM
97,425,2200,009,000400000425000,0771,00-S'03,NO SIZE,1,0113645,LANCOME
98,426,2200,009,000400000426000,0768,02-VIOLET,NO SIZE,1,0113645,LANCOME
